In [1]:
import tensorflow as tf
import numpy as np
import time

c:\anaconda3\envs\practise\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\anaconda3\envs\practise\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\anaconda3\envs\practise\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\anaconda3\envs\practise\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarnin

In [2]:
class Model:
    def __init__(self):
        with tf.variable_scope('inputs'):
            self.x = tf.placeholder(tf.float32,[None],name='input_x')
            self.y = tf.placeholder(tf.float32,[None],name='output_y')
        
    def initialize(self):
        with tf.variable_scope('hidden_layer'):
            self.W = tf.Variable([1.0], dtype=tf.float32,name='W')
            self.b = tf.Variable([1.0],dtype=tf.float32,name='b')
            
            self.y_hat = tf.add(tf.multiply(self.x,self.W,name='Wx'), self.b, name='b')
            
        print('No. of trainable variables : {}'.format(np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])))

    def add_loss(self):
        with tf.variable_scope('loss'):
            self.loss = tf.reduce_mean(tf.squared_difference(self.y, self.y_hat))
            
    def add_optimizer(self,global_step,learning_rate=1.e-3,adam_beta1=0.9,adam_beta2=0.999):
        with tf.variable_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate, adam_beta1, adam_beta2)
            gradients, variables = zip(*optimizer.compute_gradients(self.loss))
            self.gradients = gradients
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, 1.0)
            with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                self.optimize = optimizer.apply_gradients(zip(clipped_gradients, variables),global_step=global_step)

In [3]:
global_step = tf.Variable(0, name='global_step', trainable=False)

model = Model()
model.initialize()
model.add_loss()
model.add_optimizer(global_step)

Instructions for updating:
Colocations handled automatically by placer.
No. of trainable variables : 2


In [4]:
x = np.random.randn(50,)
y = 3.0 * x + 5.0

In [5]:
x

array([ 1.53942634,  1.45199584,  1.12428659, -1.17683331,  0.39828799,
        0.81691035, -2.36374788,  0.60732233, -0.12366254, -0.54987721,
       -0.15468642,  0.51275471,  0.66754244,  0.92455115, -0.09795126,
       -1.2652552 ,  0.77645818,  0.80935877, -1.03700587, -0.25661369,
       -0.92931889, -0.75452206, -0.20818565,  0.91042222, -1.04288034,
        0.96265468,  0.10041513, -0.86387517,  2.10921583, -0.50501554,
       -0.67053802,  0.0864083 , -1.21288896,  1.66420075, -0.97103521,
        0.14055298, -1.6944407 , -0.55859434, -0.12428911, -0.07062646,
       -0.3257712 , -0.26491217, -0.30052856, -0.37497409, -1.32759124,
        0.54552197, -0.9703957 , -0.90949303, -1.84386426, -0.40508839])

In [6]:
y

array([ 9.61827902,  9.35598751,  8.37285977,  1.46950007,  6.19486396,
        7.45073104, -2.09124365,  6.82196698,  4.62901238,  3.35036838,
        4.53594074,  6.53826412,  7.00262732,  7.77365346,  4.70614621,
        1.20423439,  7.32937453,  7.4280763 ,  1.8889824 ,  4.23015894,
        2.21204334,  2.73643381,  4.37544305,  7.73126666,  1.87135898,
        7.88796404,  5.30124538,  2.4083745 , 11.3276475 ,  3.48495339,
        2.98838595,  5.25922491,  1.36133311,  9.99260225,  2.08689437,
        5.42165893, -0.08332211,  3.32421698,  4.62713267,  4.78812063,
        4.02268639,  4.2052635 ,  4.09841433,  3.87507773,  1.01722628,
        6.63656591,  2.0888129 ,  2.2715209 , -0.53159278,  3.78473484])

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    start_step = sess.run(global_step)
    start_time = time.time()
    for _ in range(10000):
        step, loss, opt = sess.run([global_step, model.loss, model.optimize],feed_dict={model.x:x,model.y:y})
        
        if step % 500 == 0:
            print('Step : [%d] loss = [%f]' % (step,loss))
    print('Time taken : {} secs.'.format(time.time() - start_time))        
    print('Learnt Values : W : [%f] b : [%f]' % (sess.run(model.W), sess.run(model.b)))

Step : [500] loss = [12.851274]
Step : [1000] loss = [9.126898]
Step : [1500] loss = [6.157516]
Step : [2000] loss = [3.853718]
Step : [2500] loss = [2.127317]
Step : [3000] loss = [0.921291]
Step : [3500] loss = [0.215660]
Step : [4000] loss = [0.025458]
Step : [4500] loss = [0.001796]
Step : [5000] loss = [0.000059]
Step : [5500] loss = [0.000001]
Step : [6000] loss = [0.000000]
Step : [6500] loss = [0.000000]
Step : [7000] loss = [0.000000]
Step : [7500] loss = [0.000000]
Step : [8000] loss = [0.000000]
Step : [8500] loss = [0.000000]
Step : [9000] loss = [0.000000]
Step : [9500] loss = [0.000000]
Step : [10000] loss = [0.000000]
Time taken : 6.403891324996948 secs.
Learnt Values : W : [2.999999] b : [4.999995]
